In [1]:
import os
import json

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio import Entrez

Entrez.email = "417454220@qq.com"
datapath = "Data"

In [2]:
translated = []
startcodon = "ATG"

for record in SeqIO.parse(os.path.join(datapath, "HSP90AA1.fasta"), "fasta"):
    seqstr = str(record.seq)
    longestSeq = None
    for n in range(len(seqstr)):
        codon = seqstr[n:n+3]
        if codon == startcodon:
            newSeq = record.seq[n:].translate(to_stop=True)
            if longestSeq is None:
                longestSeq = newSeq
            elif len(longestSeq) < len(newSeq):
                longestSeq = newSeq
    translated.append(SeqRecord(
        longestSeq,
        id=record.id,
        description=record.description
    ))
    
SeqIO.write(
    sequences=translated,
    handle=os.path.join(datapath, "HSP90AA1_translated.fasta"),
    format="fasta"
)

c:\users\chengyang ji\appdata\local\programs\python\python35\lib\site-packages\Bio\Seq.py:2748: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


30

In [10]:
species = {}

for record in translated:
    seqID = record.id
    handle = Entrez.efetch(db="nucleotide", id=seqID, retmode="xml")
    (record,) = Entrez.read(handle)
    species[seqID] = record["GBSeq_source"]

In [13]:
with open(os.path.join(datapath, "seqID2species.json"), 'w') as f:
    json.dump(species, f, indent=2)